In [ ]:
import cv2
import numpy as np

img = cv2.imread("digits.png", cv2.IMREAD_GRAYSCALE)
print(img.shape)

(1000, 2000)


In [ ]:
cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]
x = np.array(cells)
print(x.shape)

(50, 100, 20, 20)


In [ ]:
X = x.reshape(-1, 400)
y = np.repeat(np.arange(10), 500)

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes = (512, 256, 128), verbose=True)
model.fit(train_X, train_y)

Iteration 1, loss = 9.20565264
Iteration 2, loss = 1.26573983
Iteration 3, loss = 0.46351297
Iteration 4, loss = 0.20101019
Iteration 5, loss = 0.10167266
Iteration 6, loss = 0.04124762
Iteration 7, loss = 0.01889318
Iteration 8, loss = 0.01213614
Iteration 9, loss = 0.00378833
Iteration 10, loss = 0.00113555
Iteration 11, loss = 0.00074657
Iteration 12, loss = 0.00055216
Iteration 13, loss = 0.00050901
Iteration 14, loss = 0.00046670
Iteration 15, loss = 0.00044352
Iteration 16, loss = 0.00042695
Iteration 17, loss = 0.00041355
Iteration 18, loss = 0.00040018
Iteration 19, loss = 0.00039028
Iteration 20, loss = 0.00037976
Iteration 21, loss = 0.00037167
Iteration 22, loss = 0.00036499
Iteration 23, loss = 0.00035910
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(hidden_layer_sizes=(512, 256, 128), verbose=True)

In [ ]:
model.score(test_X, test_y)

0.9106666666666666

In [ ]:
# img4 = cv2.imread("four.png", 0) # 0은 cv2.IMREAD_GRAYSCALE과 같은 의미이다.

In [ ]:
# pred = model.predict_proba(img4.reshape(-1, 400))
# print(pred)

In [ ]:
# np.argmax(pred)

In [ ]:
# model.predict(img4.reshape(-1, 400))

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
if cap.isOpened() :
    while True :
        ret, img = cap.read()
        if not ret: break

        g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, bin_img = cv2.threshold(g_img, 110, 225, cv2.THRESH_BINARY_INV)
        contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours :
            (x, y), radius = cv2.minEnclosingCircle(contour)
            if radius < 5 : continue # 5보다 작은 객체 Skip
            xs, xe = int(x-radius), int(x+radius)
            ys, ye = int(y-radius), int(y+radius)
            cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1)
            roi = bin_img[ys:ye, xs:xe]
            if roi.shape[0]*roi.shape[1] == 0: continue
            roi = cv2.resize(roi, dsize=(16, 16))
            A = np.zeros((20, 20))
            A[2:-2, 2:-2] = roi[:, :]
            A = A.reshape(-1, 400)
            num = model.predict(A)
            cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200, 0, 0))
        cv2.imshow("Image", bin_img)
        if cv2.waitKey(1)&0xFF == 27:break # ESC
else :
    print("Camera not opened")
cap.release()
cv2.destroyAllWindows()